# Итоговый проект по курсу "Рекомендательные системы"

In [1]:
import pandas as pd
import numpy as np

import math

# Метрики
from best_rec_lib.metrics import precision_at_k, recall_at_k, ap_k

# Префильтрация, работа с фичами, предсказания
from best_rec_lib.utils import trainValLvl1Split, prefilter_items, trainTestDf, getRecommendationsLvl2

# Класс рекомендера
from best_rec_lib.recommenders import MainRecommender

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


import warnings
warnings.filterwarnings("ignore")

### Загрузка данных

In [2]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')
data_test = pd.read_csv('retail_test.csv')

In [3]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

**Делим данные на трейнировочную и валидационную выборки для модели 1го уровня**

In [4]:
test_user_ids = list(set(data_test.user_id))

# Для иодели 1го уровння
data_train_lvl_1, data_val_lvl_1 = trainValLvl1Split(data, test_user_ids)

# Для иодели 2го уровння
data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = data_test.copy()

**Prefilter items**

In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=6000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Количество item сокращено с {} до {}'.format(n_items_before, n_items_after))

Количество item сокращено с 78673 до 6001


**Модель 1го уровня**

In [6]:
%%time
recommender = MainRecommender(data_train_lvl_1)


Wall time: 16.1 s


**Cписки рекомендаций для пользователей**

In [ ]:
%%time
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

# Получим предсказания. \
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

**Проверка метрики модели 1го уровня**

In [ ]:
# На валидационном датасете lvl 1
result = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result = result.merge(users_lvl_2, on='user_id', how='left')
result.loc[result['candidates'].isnull(), 'candidates'] = result.loc[result['candidates'].isnull(), 'user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [ ]:
print('Precision@k: ', result.apply(lambda row: precision_at_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('MAP@k: ', result.apply(lambda row: ap_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('Recall@k: ', result.apply(lambda row: recall_at_k(row['candidates'], row['actual'], 500), axis=1).mean())

In [ ]:
# На тестовом датасете lvl 2
result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result = result.merge(users_lvl_2, on='user_id', how='left')
result.loc[result['candidates'].isnull(), 'candidates'] = result.loc[result['candidates'].isnull(), 'user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [ ]:
print('Precision@k: ', result.apply(lambda row: precision_at_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('MAP@k: ', result.apply(lambda row: ap_k(row['candidates'], row['actual'], 5), axis=1).mean())
print('Recall@k: ', result.apply(lambda row: recall_at_k(row['candidates'], row['actual'], 500), axis=1).mean())

**Датафрейм для обучения модели 2го уровня**

In [ ]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [ ]:
%%time
# Длоя создания доп. фичей использую data_train_lvl_1. 
X_train, y_train = trainTestDf(data_train_lvl_1, targets_lvl_2, item_features, user_features)

**Обучение модели классификации и получение предсказаний**

In [ ]:
preds_lvl_2 = targets_lvl_2[['user_id', 'item_id', 'target']]

In [ ]:
%%time

# Рекомендации CatBoost
preds_lvl_2['pred_proba'] = recommender.get_CatBoost_lvl2_preds(X_train, y_train)

# Рекомендации LGBM
# preds_lvl_2['pred_proba'] = recommender.get_LGBM_lvl2_preds(X_train, y_train)

In [ ]:
result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [ ]:
test_users = result.shape[0]
new_test_users = len(set(result['user_id']) - set(preds_lvl_2['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))
new_test_users = list(set(result['user_id']) - set(preds_lvl_2['user_id']))

In [ ]:
# Для каждого пользователя даем 5 уникальных (!!!) рекомендаций.
result['classification_rec'] = result['user_id'].map(lambda x: getRecommendationsLvl2(preds_lvl_2.loc[preds_lvl_2['user_id'] == x], recommender, N=5))

**Рассчет метрики результата работы моделей lvl1 + lvl2**

In [ ]:
print('Precision@k: ', result.apply(lambda row: precision_at_k(row['classification_rec'], row['actual'], 5), axis=1).mean())
print('MAP@k: ', result.apply(lambda row: ap_k(row['classification_rec'], row['actual'], 5), axis=1).mean())
print('Recall@k: ', result.apply(lambda row: recall_at_k(row['classification_rec'], row['actual'], 5), axis=1).mean())

**СОхранение предсказаний**

In [ ]:
# Столбцы: user_id, actual, classification_rec
result.to_csv('predictions.csv', index=False)